# Calculate distance to Facilities and add to Listings

**Note the osm data and the listings data have both been transformed to have the same coordinate reference system "ESPG:28992"**

In [2]:
import geopandas as gpd
from shapely.ops import nearest_points
from shapely.geometry import Point, LineString

C:\Users\hassa\AppData\Local\Temp\ipykernel_11344\3147119002.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [10]:
# Read in the shapefiles
airbnb = gpd.read_file('Data_2.0/listings.shp')

# Read in file with the point data 
restaurants = gpd.read_file('Data_2.0/Restaurants.shp')
nightclubs = gpd.read_file('Data_2.0/nightclubs.shp')
Suppermarkets = gpd.read_file('Data_2.0/Suppermarkets.shp')
parks_osm = gpd.read_file('Data_2.0/parks_osm.shp')

#Travel
train_stations = gpd.read_file('Data_2.0/train_stations.shp')
bus_stops = gpd.read_file('Data_2.0/bus_stops.shp')
bicycle_rental = gpd.read_file('Data_2.0/bicycle_rental.shp')

#Create list of amenity_df
ls_amenity_df = [restaurants, nightclubs, Suppermarkets, parks_osm, train_stations, bus_stops, bicycle_rental]

#amenity name
amenity_names = ["restaurant", "nightclub", "Suppermarket", "park", "train_station", "bus_stops", "bicycle_rental"]

In [11]:
import geopandas as gpd
from shapely.ops import nearest_points
from shapely.geometry import Point, MultiPoint, LineString

def calculate_nearest_amenities(airbnb_df, amenity_dfs, amenity_names):
    # Loop through each amenity dataframe and amenity name
    for amenity_df, amenity_name in zip(amenity_dfs, amenity_names):
        # Create an empty column in the Airbnb dataframe to store the distances
        airbnb_df[f'distance_to_{amenity_name}'] = None
        
        # Loop through each Airbnb listing
        for i, airbnb_row in airbnb_df.iterrows():
            # Create a shapely point object from the Airbnb listing's geometry
            airbnb_point = airbnb_row.geometry
            
            # Calculate the distance between the Airbnb listing and the nearest amenity
            nearest_amenity_points = nearest_points(airbnb_point, amenity_df.geometry.unary_union)
            if isinstance(nearest_amenity_points[1], Point):
                # If the nearest amenity has only one point, calculate the distance between the Airbnb listing and that point
                distance = LineString([airbnb_point, nearest_amenity_points[1]]).length
            else:
                # If the nearest amenity has multiple points, calculate the distance between the Airbnb listing and the nearest
                # point of the nearest amenity
                nearest_point = nearest_points(airbnb_point, MultiPoint(nearest_amenity_points))[1]
                distance = LineString([airbnb_point, nearest_point]).length
            
            # Store the distance in the 'distance_to_{amenity_name}' column
            airbnb_df.at[i, f'distance_to_{amenity_name}'] = distance
    
    return airbnb_df

In [12]:
airbnb_df_amenities = calculate_nearest_amenities(airbnb, ls_amenity_df, amenity_names)

In [13]:
airbnb_df_amenities.head()

,id,listing_ur,scrape_id,last_scrap,source,name,descriptio,neighborho,picture_ur,host_id,...,field_79,field_80,geometry,distance_to_restaurant,distance_to_nightclub,distance_to_Suppermarket,distance_to_park,distance_to_train_station,distance_to_bus_stops,distance_to_bicycle_rental
0,528022,https://www.airbnb.com/rooms/528022,20221205143541,2022-12-05,city scrape,Somewhere Else,Highlights<br />*Located only 5 minutes from C...,None,https://a0.muscache.com/pictures/bf6fa79c-5863...,2594559,...,None,None,POINT (121049.635 489371.211),117.842544,926.202216,631.183065,346.810401,1575.944918,272.335519,840.656392
1,2818,https://www.airbnb.com/rooms/2818,20221205143541,2022-12-05,city scrape,Quiet Garden View Room & Super Fast Wi-Fi,Quiet Garden View Room & Super Fast Wi-Fi<br /...,"Indische Buurt (""Indies Neighborhood"") is a ne...",https://a0.muscache.com/pictures/10272854/8dcc...,3159,...,None,None,POINT (124783.464 486366.064),230.922795,1200.228526,323.632332,510.420063,930.007725,231.353635,539.058052
2,20168,https://www.airbnb.com/rooms/20168,20221205143541,2022-12-05,previous scrape,Studio with private bathroom in the centre 1,17th century Dutch townhouse in the heart of t...,Located just in between famous central canals....,https://a0.muscache.com/pictures/69979628/fd6a...,59484,...,None,None,POINT (121401.482 486356.768),140.691875,203.391798,180.047576,588.015693,423.879518,247.540031,290.933778
3,27886,https://www.airbnb.com/rooms/27886,20221205143541,2022-12-05,city scrape,"Romantic, stylish B&B houseboat in canal district",Stylish and romantic houseboat on fantastic hi...,"Central, quiet, safe, clean and beautiful.",https://a0.muscache.com/pictures/02c2da9d-660e...,97647,...,None,None,POINT (121279.753 488976.855),61.19738,843.206587,611.931839,223.284618,1119.700723,161.951809,384.107011
4,28871,https://www.airbnb.com/rooms/28871,20221205143541,2022-12-05,city scrape,Comfortable double room,<b>The space</b><br />In a monumental house ri...,"Flower market , Leidseplein , Rembrantsplein",https://a0.muscache.com/pictures/160889/362340...,124245,...,None,None,POINT (121199.267 486767.616),30.732054,58.740119,256.434528,877.595509,785.871827,206.054152,332.740755


In [14]:
airbnb_df_amenities.crs

<Derived Projected CRS: EPSG:28992>
Name: Amersfoort / RD New
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Oblique Stereographic
Datum: Amersfoort
- Ellipsoid: Bessel 1841
- Prime Meridian: Greenwich

In [ ]:
#airbnb_df_amenities.to_csv("Data/airbnb_df_amenities.csv")